In [2]:
import pandas as pd
import numpy as np 

pd.set_option('display.max_columns', 300)

In [3]:
# importing the data I will be predicting on. Note that prices are not included here.
df = pd.read_csv('Data/CSV/kc_house_data_test_features.csv', index_col=0)
print(df.shape)
df.head()

(4322, 20)


id             date  bedrooms  bathrooms  sqft_living  sqft_lot  \
0  1974300020  20140827T000000         4       2.50         2270     11500   
1  1974300020  20150218T000000         4       2.50         2270     11500   
2  3630020380  20141107T000000         3       2.50         1470      1779   
3  1771000290  20141203T000000         3       1.75         1280     16200   
4  5126310470  20150115T000000         4       2.75         2830      8126   

   floors  waterfront  view  condition  grade  sqft_above  sqft_basement  \
0     1.0           0     0          3      8        1540            730   
1     1.0           0     0          3      8        1540            730   
2     2.0           0     0          3      8        1160            310   
3     1.0           0     0          3      8        1030            250   
4     2.0           0     0          3      8        2830              0   

   yr_built  yr_renovated  zipcode      lat     long  sqft_living15  \
0      1967             0    98034  47.7089 -122.241           2020   
1      1967             0    98034  47.7089 -122.241           2020   
2      2005             0    98029  47.5472 -121.998           1470   
3      1976             0    98077  47.7427 -122.071           1160   
4      2005             0    98059  47.4863 -122.140           2830   

   sqft_lot15  
0       10918  
1       10918  
2        1576  
3       10565  
4        7916

In [4]:
# importing my picckled model 
import pickle
infile = open("Data/pickles/model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [5]:
#importing my pickled list
infile = open("Data/pickles/stuff_things.pickle",'rb')
stuff_things = pickle.load(infile)
infile.close()

In [6]:
# checking out my pickled list
stuff_things

[2,
 3,
 Index(['sqft_living', 'sqft_basement', 'waterfront', 'SqFpR', 'sqft_lot',
        'bsmt', 'grade', 'zipcode_98001', 'zipcode_98002', 'zipcode_98003',
        'zipcode_98004', 'zipcode_98005', 'zipcode_98006', 'zipcode_98007',
        'zipcode_98008', 'zipcode_98010', 'zipcode_98011', 'zipcode_98014',
        'zipcode_98019', 'zipcode_98022', 'zipcode_98023', 'zipcode_98024',
        'zipcode_98027', 'zipcode_98028', 'zipcode_98029', 'zipcode_98030',
        'zipcode_98031', 'zipcode_98032', 'zipcode_98033', 'zipcode_98034',
        'zipcode_98038', 'zipcode_98039', 'zipcode_98040', 'zipcode_98042',
        'zipcode_98045', 'zipcode_98052', 'zipcode_98053', 'zipcode_98055',
        'zipcode_98056', 'zipcode_98058', 'zipcode_98059', 'zipcode_98065',
        'zipcode_98070', 'zipcode_98072', 'zipcode_98074', 'zipcode_98075',
        'zipcode_98077', 'zipcode_98092', 'zipcode_98102', 'zipcode_98103',
        'zipcode_98105', 'zipcode_98106', 'zipcode_98107', 'zipcode_98108',
     

In [7]:
#checking for missing values
df.isna().mean()

id               0.0
date             0.0
bedrooms         0.0
bathrooms        0.0
sqft_living      0.0
sqft_lot         0.0
floors           0.0
waterfront       0.0
view             0.0
condition        0.0
grade            0.0
sqft_above       0.0
sqft_basement    0.0
yr_built         0.0
yr_renovated     0.0
zipcode          0.0
lat              0.0
long             0.0
sqft_living15    0.0
sqft_lot15       0.0
dtype: float64

In [8]:
#applying the same cleaning and features from my test model to this data set
clean_dict_bathrooms = {0: stuff_things[0],
                        0.5:stuff_things[0]}

clean_dict_rooms = {0: stuff_things[1]}

df['sale_quarter'] = pd.DatetimeIndex(df['date']).month //4 +1
df['bedrooms']= np.where(df['bedrooms'] >11, 10, df['bedrooms'])
df['bathrooms']= np.where(df['bathrooms'] >8, 7, df['bathrooms'])


df['bedrooms'] = df['bedrooms'].replace(clean_dict_rooms)
df['bathrooms'] = df['bathrooms'].replace(clean_dict_bathrooms)
df['bedrooms'].replace(33,3, inplace = True)
df['bsmt'] = np.where(df['sqft_basement']>0, 1, 0)

df['SqFpR'] = df['sqft_living'] / (df['bedrooms'] + df['bathrooms'])
df['yard'] = df['sqft_lot'] - (df['sqft_living']/df['floors'])
df['sale_quarter'] = pd.DatetimeIndex(df['date']).month //4 +1
df['new_condition'] = (df['condition']) //2 +1  
for label, row in df.iterrows():
    if row['yr_renovated'] ==0:
        df.loc[label,'YSLC'] = 2021 - row['yr_built']
    else: 
        df.loc[label,'YSLC'] = 2021 - row['yr_renovated'] 


In [9]:
# making sure the columns I feed into this model are the same from my test model
final_columns = list(stuff_things[2])

In [10]:
final_columns

['sqft_living',
 'sqft_basement',
 'waterfront',
 'SqFpR',
 'sqft_lot',
 'bsmt',
 'grade',
 'zipcode_98001',
 'zipcode_98002',
 'zipcode_98003',
 'zipcode_98004',
 'zipcode_98005',
 'zipcode_98006',
 'zipcode_98007',
 'zipcode_98008',
 'zipcode_98010',
 'zipcode_98011',
 'zipcode_98014',
 'zipcode_98019',
 'zipcode_98022',
 'zipcode_98023',
 'zipcode_98024',
 'zipcode_98027',
 'zipcode_98028',
 'zipcode_98029',
 'zipcode_98030',
 'zipcode_98031',
 'zipcode_98032',
 'zipcode_98033',
 'zipcode_98034',
 'zipcode_98038',
 'zipcode_98039',
 'zipcode_98040',
 'zipcode_98042',
 'zipcode_98045',
 'zipcode_98052',
 'zipcode_98053',
 'zipcode_98055',
 'zipcode_98056',
 'zipcode_98058',
 'zipcode_98059',
 'zipcode_98065',
 'zipcode_98070',
 'zipcode_98072',
 'zipcode_98074',
 'zipcode_98075',
 'zipcode_98077',
 'zipcode_98092',
 'zipcode_98102',
 'zipcode_98103',
 'zipcode_98105',
 'zipcode_98106',
 'zipcode_98107',
 'zipcode_98108',
 'zipcode_98109',
 'zipcode_98112',
 'zipcode_98115',
 'zipcode

In [11]:
#applying the same dumming of variables with the same subset of the test model
df = df[['sale_quarter','sqft_living', 'sqft_basement','view','grade', 'waterfront','zipcode','new_condition','SqFpR','sqft_lot','bsmt' ]]


df = pd.get_dummies(df, columns=['zipcode']).drop(['zipcode_98117'], axis = 1)
df = pd.get_dummies(df, columns=['new_condition']).drop(['new_condition_2'], axis = 1)
df = pd.get_dummies(df, columns=['view']).drop(['view_2'], axis = 1)
df = pd.get_dummies(df, columns=['sale_quarter']).drop(['sale_quarter_3'], axis = 1)

In [12]:
#checking out the subset frame
df[final_columns]

sqft_living  sqft_basement  waterfront       SqFpR  sqft_lot  bsmt  \
0            2270            730           0  349.230769     11500     1   
1            2270            730           0  349.230769     11500     1   
2            1470            310           0  267.272727      1779     1   
3            1280            250           0  269.473684     16200     1   
4            2830              0           0  419.259259      8126     0   
...           ...            ...         ...         ...       ...   ...   
4318         1530              0           0  278.181818      1131     0   
4319         2310              0           0  355.384615      5813     0   
4320         1020              0           0  370.909091      1350     0   
4321         1600              0           0  290.909091      2388     0   
4322         1020              0           0  370.909091      1076     0   

      grade  zipcode_98001  zipcode_98002  zipcode_98003  zipcode_98004  \
0         8              0              0              0              0   
1         8              0              0              0              0   
2         8              0              0              0              0   
3         8              0              0              0              0   
4         8              0              0              0              0   
...     ...            ...            ...            ...            ...   
4318      8              0              0              0              0   
4319      8              0              0              0              0   
4320      7              0              0              0              0   
4321      8              0              0              0              0   
4322      7              0              0              0              0   

      zipcode_98005  zipcode_98006  zipcode_98007  zipcode_98008  \
0                 0              0              0              0   
1                 0              0              0              0   
2                 0              0              0              0   
3                 0              0              0              0   
4                 0              0              0              0   
...             ...            ...            ...            ...   
4318              0              0              0              0   
4319              0              0              0              0   
4320              0              0              0              0   
4321              0              0              0              0   
4322              0              0              0              0   

      zipcode_98010  zipcode_98011  zipcode_98014  zipcode_98019  \
0                 0              0              0              0   
1                 0              0              0              0   
2                 0              0              0              0   
3                 0              0              0              0   
4                 0              0              0              0   
...             ...            ...            ...            ...   
4318              0              0              0              0   
4319              0              0              0              0   
4320              0              0              0              0   
4321              0              0              0              0   
4322              0              0              0              0   

      zipcode_98022  zipcode_98023  zipcode_98024  zipcode_98027  \
0                 0              0              0              0   
1                 0              0              0              0   
2                 0              0              0              0   
3                 0              0              0              0   
4                 0              0              0              0   
...             ...            ...            ...            ...   
4318              0              0              0              0   
4319          

In [13]:
#saving my final model to a variable
final_answers = model.predict(df[final_columns])

In [14]:
# The length is correct!
len(final_answers)

4322

In [15]:
#changing my model array into a data frame
df_final = pd.DataFrame(final_answers)

In [16]:
# Saving that data frame to a CSV
df_final.to_csv('Data/CSV/housing_preds_Tyrell_Jackson.csv')